In [1]:
import arviz as az
from arviz.data import convert_to_dataset
from arviz.stats.diagnostics import bfmi,_bfmi,geweke as ge,rhat as rh,_ess as es
from arviz.stats.diagnostics import _rhat,_split_chains,_z_scale
from arviz.stats.diagnostics import _rhat_rank as rk
from arviz.stats.diagnostics import ks_summary as ks
from arviz.stats.diagnostics import _mc_error as me, _mcse_mean, _mcse_quantile, _mcse_sd as msd,_ess_mean,_ess_sd
from arviz.utils import conditional_jit
from arviz.stats.stats_utils import not_valid as _not_valid, autocov as _autocov
import numpy as np
from line_profiler import LineProfiler
import numba
import pandas as pd
from scipy.fftpack import next_fast_len
from scipy.stats.mstats import mquantiles
from scipy import stats
from xarray import apply_ufunc
import xarray as xr
import warnings

In [2]:
data = np.random.randn(1000,10000)

In [3]:
%timeit bfmi(data)

196 ms ± 3.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
lp = LineProfiler()
wrapper = lp(bfmi)
wrapper(data)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.208983 s
File: /home/banzee/Desktop/arviz/arviz/stats/diagnostics.py
Function: bfmi at line 24

Line #      Hits         Time  Per Hit   % Time  Line Contents
    24                                           def bfmi(data):
    25                                               r"""Calculate the estimated Bayesian fraction of missing information (BFMI).
    26                                           
    27                                               BFMI quantifies how well momentum resampling matches the marginal energy distribution. For more
    28                                               information on BFMI, see https://arxiv.org/pdf/1604.00695v1.pdf. The current advice is that
    29                                               values smaller than 0.3 indicate poor sampling. However, this threshold is provisional and may
    30                                               change. See http://mc-stan.org/users/documentation/case-studies/py

In [5]:
lp = LineProfiler()
wrapper = lp(_bfmi)
wrapper(data)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.203059 s
File: /home/banzee/Desktop/arviz/arviz/stats/diagnostics.py
Function: _bfmi at line 547

Line #      Hits         Time  Per Hit   % Time  Line Contents
   547                                           def _bfmi(energy):
   548                                               r"""Calculate the estimated Bayesian fraction of missing information (BFMI).
   549                                           
   550                                               BFMI quantifies how well momentum resampling matches the marginal energy distribution. For more
   551                                               information on BFMI, see https://arxiv.org/pdf/1604.00695v1.pdf. The current advice is that
   552                                               values smaller than 0.3 indicate poor sampling. However, this threshold is provisional and may
   553                                               change. See http://mc-stan.org/users/documentation/case-studi

In [6]:
@numba.njit
def _var_1d(data):
    a,b = 0,0
    for i in data:
        a = a+i
        b = b+i*i
    return b/len(data)-((a/len(data))**2)

@numba.njit
def _var_2d(data):
    a,b = data.shape
    var = np.zeros(a)
    for i in range(0,a):
        var[i] = _var_1d(data[i,:])
    return var


def _bfmi_jit(energy):
    energy_mat = np.atleast_2d(energy)
    if energy_mat.ndim==2:
        num = np.square(np.diff(energy_mat, axis=1)).mean(axis=1)  # pylint: disable=no-member
        den = _var_2d(energy_mat)
    return num / den

def bfmi_new(data):
    if isinstance(data, np.ndarray):
        return _bfmi_jit(data)

    dataset = convert_to_dataset(data, group="sample_stats")
    if not hasattr(dataset, "energy"):
        raise TypeError("Energy variable was not found.")
    return _bfmi_jit(dataset.energy)



In [7]:
%timeit bfmi_new(data)

122 ms ± 3.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit az.stats.diagnostics.bfmi(data)

197 ms ± 6.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit bfmi_new(np.random.randn(1000000))

81.4 ms ± 6.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit az.bfmi(np.random.randn(1000000))

82.8 ms ± 6.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit bfmi_new(az.load_arviz_data("centered_eight"))

99.1 ms ± 991 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit az.bfmi(az.load_arviz_data("centered_eight"))

111 ms ± 6.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
'''Much better improvement on larger datasets. A gain on a few milliseconds on school'''

'Much better improvement on larger datasets. A gain on a few milliseconds on school'

In [14]:
"""""""""""""""""""""""""""""""""""""""""""""""""ks_summary"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

'ks_summary'

In [15]:
%autosave 0

Autosave disabled


In [16]:
data = np.random.randn(10_00_00,1000)
school  = az.load_arviz_data("centered_eight").posterior["mu"].values

ks_summary(data)

In [17]:
lp = LineProfiler()
wrapper = lp(ks)
wrapper(data)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 11.7259 s
File: /home/banzee/Desktop/arviz/arviz/stats/diagnostics.py
Function: ks_summary at line 520

Line #      Hits         Time  Per Hit   % Time  Line Contents
   520                                           def ks_summary(pareto_tail_indices):
   521                                               """Display a summary of Pareto tail indices.
   522                                           
   523                                               Parameters
   524                                               ----------
   525                                               pareto_tail_indices : array
   526                                                 Pareto tail indices.
   527                                           
   528                                               Returns
   529                                               -------
   530                                               df_k : dataframe
   531                                 

In [18]:
'''bottleneck ar np.historgram'''

'bottleneck ar np.historgram'

In [19]:
@conditional_jit
def _histogram(data):
    kcounts, _ = np.histogram(data,bins=[-np.Inf, 0.5, 0.7, 1, np.Inf])
    return kcounts


def ks_summary_new(pareto_tail_indices):
    kcounts = _histogram(pareto_tail_indices)
    kprop = kcounts / len(pareto_tail_indices) * 100
    df_k = pd.DataFrame(
        dict(_=["(good)", "(ok)", "(bad)", "(very bad)"], Count=kcounts, Pct=kprop)
    ).rename(index={0: "(-Inf, 0.5]", 1: " (0.5, 0.7]", 2: "   (0.7, 1]", 3: "   (1, Inf)"})

    if np.sum(kcounts[1:]) == 0:
        warnings.warn("All Pareto k estimates are good (k < 0.5)")
    elif np.sum(kcounts[2:]) == 0:
        warnings.warn("All Pareto k estimates are ok (k < 0.7)")

    return df_k

In [20]:
ks_summary_new(data)==ks(data)

,_,Count,Pct
"(-Inf, 0.5]",True,True,True
"(0.5, 0.7]",True,True,True
"(0.7, 1]",True,True,True
"(1, Inf)",True,True,True


In [21]:
%timeit ks_summary_new(data)

1.37 s ± 4.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit ks(data)

11.2 s ± 14.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit ks_summary_new(school)

1.46 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%timeit ks(school)

1.95 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%timeit ks_summary_new(np.random.randn(10000000))

832 ms ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit ks(np.random.randn(10000000))

1.84 s ± 46.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
'''PHEW. 10 times faster on large datasets. Much better performance on every dataset'''

'PHEW. 10 times faster on large datasets. Much better performance on every dataset'

In [28]:
"""""""""""""""""""""""""""""""""""""""""""""""geweke"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

'""geweke'

In [29]:
data = np.random.randn(10000000)
school = az.load_arviz_data("centered_eight").posterior["mu"].values
school = school[1,:]

In [30]:
lp = LineProfiler()
wrapper = lp(ge)
wrapper(data,0.1,0.5,200)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 8.12892 s
File: /home/banzee/Desktop/arviz/arviz/stats/diagnostics.py
Function: geweke at line 448

Line #      Hits         Time  Per Hit   % Time  Line Contents
   448                                           def geweke(ary, first=0.1, last=0.5, intervals=20):
   449                                               r"""Compute z-scores for convergence diagnostics.
   450                                           
   451                                               Compare the mean of the first % of series with the mean of the last % of series. x is divided
   452                                               into a number of segments for which this difference is computed. If the series is converged,
   453                                               this score should oscillate between -1 and 1.
   454                                           
   455                                               Parameters
   456                                      

In [31]:
@conditional_jit
def _var_1d(data):
    a,b = 0,0
    for i in data:
        a = a+i
        b = b+i*i
    return b/len(data)-((a/len(data))**2)


@numba.vectorize(nopython=True)   ##Remember to make a conditional_vectorize function
def _sqr(a,b):
    return np.sqrt(a+b)


@conditional_jit
def geweke_new(ary, first=0.1, last=0.5, intervals=20):
    # Filter out invalid intervals
    for interval in (first, last):
        if interval <= 0 or interval >= 1:
            raise ValueError("Invalid intervals for Geweke convergence analysis", (first, last))
    if first + last >= 1:
        raise ValueError("Invalid intervals for Geweke convergence analysis", (first, last))

    # Initialize list of z-scores
    zscores = []

    # Last index value
    end = len(ary) - 1

    # Start intervals going up to the <last>% of the chain
    last_start_idx = (1 - last) * end

    # Calculate starting indices
    start_indices = np.linspace(0, last_start_idx, num=intervals, endpoint=True, dtype=int)

    # Loop over start indices
    for start in start_indices:
        # Calculate slices
        first_slice = ary[start : start + int(first * (end - start))]
        last_slice = ary[int(end - last * (end - start)) :]

        z_score = first_slice.mean() - last_slice.mean()
        D = _sqr(_var_1d(first_slice), _var_1d(last_slice))
        z_score = z_score/D

        zscores.append([start, z_score])

    return np.array(zscores)



In [32]:
%timeit geweke_new(data)

236 ms ± 3.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%timeit az.geweke(data)

737 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit geweke_new(data,intervals=300)

3.82 s ± 195 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit az.geweke(data,intervals=300)

10.4 s ± 395 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
np.allclose(geweke_new(data,intervals=500), az.geweke(data,intervals=500))

True

In [37]:
%timeit geweke_new(school)

1.54 ms ± 150 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [38]:
%timeit az.geweke(school)

2.75 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [39]:
"""""""""""""""""""""""""""""""""""""""""GWEKE WORKS WELL"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

'""GWEKE WORKS WELL'

In [40]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""ess"""""""""""""""""""""""""""""""""""""""""""""""""""""""""

'ess'

In [41]:
numpy_data = np.random.randn(1000,1000)
dict_data = {"posterior":numpy_data}

In [42]:
%timeit az.ess(numpy_data)

555 ms ± 9.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%timeit az.ess(dict_data)

567 ms ± 24.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
lp = LineProfiler()
wrapper = lp(az.ess)
wrapper(numpy_data)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.572761 s
File: /home/banzee/Desktop/arviz/arviz/stats/diagnostics.py
Function: ess at line 132

Line #      Hits         Time  Per Hit   % Time  Line Contents
   132                                           def ess(data, *, var_names=None, method="bulk", relative=False, prob=None):
   133                                               r"""Calculate estimate of the effective sample size.
   134                                           
   135                                               Parameters
   136                                               ----------
   137                                               data : obj
   138                                                   Any object that can be converted to an az.InferenceData object.
   139                                                   Refer to documentation of az.convert_to_dataset for details.
   140                                                   For ndarray: shape = (chain, draw).
 

In [45]:
lp = LineProfiler()
wrapper = lp(az.ess)
wrapper(dict_data)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.602527 s
File: /home/banzee/Desktop/arviz/arviz/stats/diagnostics.py
Function: ess at line 132

Line #      Hits         Time  Per Hit   % Time  Line Contents
   132                                           def ess(data, *, var_names=None, method="bulk", relative=False, prob=None):
   133                                               r"""Calculate estimate of the effective sample size.
   134                                           
   135                                               Parameters
   136                                               ----------
   137                                               data : obj
   138                                                   Any object that can be converted to an az.InferenceData object.
   139                                                   Refer to documentation of az.convert_to_dataset for details.
   140                                                   For ndarray: shape = (chain, draw).
 

In [46]:
def autocov(ary, axis=-1):
    axis = axis if axis > 0 else len(ary.shape) + axis
    n = ary.shape[axis]
    m = next_fast_len(2 * n)
    ary = ary - ary.mean(axis, keepdims=True)

    # added to silence tuple warning for a submodule
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        ifft_ary = np.fft.rfft(ary, n=m, axis=axis)
        ifft_ary *= np.conjugate(ifft_ary)

        shape = tuple(
            slice(None) if dim_len != axis else slice(0, n) for dim_len, _ in enumerate(ary.shape)
        )
        cov = np.fft.irfft(ifft_ary, n=m, axis=axis)[shape]
        cov /= n

    return cov


In [47]:
autocov(numpy_data,axis=1).shape

(1000, 1000)

In [48]:
lp = LineProfiler()
wrapper = lp(az.autocov)
wrapper(numpy_data)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.052147 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats_utils.py
Function: autocov at line 17

Line #      Hits         Time  Per Hit   % Time  Line Contents
    17                                           def autocov(ary, axis=-1):
    18                                               """Compute autocovariance estimates for every lag for the input array.
    19                                           
    20                                               Parameters
    21                                               ----------
    22                                               ary : Numpy array
    23                                                   An array containing MCMC samples
    24                                           
    25                                               Returns
    26                                               -------
    27                                               acov: Numpy array same size as the inpu

In [49]:
'''Bottleneck :: np.fft.rfft and np.conjugate'''

'Bottleneck :: np.fft.rfft and np.conjugate'

In [50]:
def _fft(x,m,axis):
    return np.fft.rfft(x,n=m,axis=axis)


@numba.jit
def _fft_new(x,m,axis):
    N =  np.fft.rfft(x,n=m,axis=axis)
    return N

def conjuc(data):
    return np.conjugate(data)


@numba.vectorize
def nconjuc(data):
    return np.conjugate(data)

In [51]:
%timeit _fft(numpy_data,2000,1)

12.7 ms ± 669 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
%timeit _fft_new(numpy_data,2000,1)

12.6 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
#Jitting fft is of no use

In [54]:
%timeit conjuc(numpy_data)

2.92 ms ± 263 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
np.conjugate([1,2,3])

array([1, 2, 3])

In [56]:
%timeit nconjuc(numpy_data)

3.39 ms ± 568 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [57]:
def autocov_new(ary, axis=-1):
    axis = axis if axis > 0 else len(ary.shape) + axis
    n = ary.shape[axis]
    m = next_fast_len(2 * n)
    ary = ary - ary.mean(axis, keepdims=True)

    # added to silence tuple warning for a submodule
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        ifft_ary = np.fft.rfft(ary, n=m, axis=axis)
        ifft_ary *= nconjuc(ifft_ary)

        shape = tuple(
            slice(None) if dim_len != axis else slice(0, n) for dim_len, _ in enumerate(ary.shape)
        )
        cov = np.fft.irfft(ifft_ary, n=m, axis=axis)[shape]
        cov /= n

    return cov


In [58]:
%timeit autocov_new(numpy_data)

44.4 ms ± 5.63 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
%timeit (az.autocov(numpy_data))

44.3 ms ± 3.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [60]:
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

''

In [61]:
numpy_data = np.random.randn(1000,1000)
dict_data = {"posterior":numpy_data}

In [62]:
def _z_scale(ary):
    ary = np.asarray(ary)
    size = ary.size
    rank = stats.rankdata(ary, method="average")
    z = stats.norm.ppf((rank - 0.5) / size)
    z = z.reshape(ary.shape)
    return z

In [63]:
lp = LineProfiler()
wrapper = lp(_z_scale)
wrapper(numpy_data)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.552019 s
File: <ipython-input-62-21c5aaeb33f5>
Function: _z_scale at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def _z_scale(ary):
     2         1          9.0      9.0      0.0      ary = np.asarray(ary)
     3         1          3.0      3.0      0.0      size = ary.size
     4         1     354677.0 354677.0     64.3      rank = stats.rankdata(ary, method="average")
     5         1     197317.0 197317.0     35.7      z = stats.norm.ppf((rank - 0.5) / size)
     6         1         12.0     12.0      0.0      z = z.reshape(ary.shape)
     7         1          1.0      1.0      0.0      return z



In [64]:
@numba.njit
def summ(x):
    return x.cumsum()

In [65]:
def rankdata_new(arr):
    arr = np.ravel(arr)
    sorter = np.argsort(arr, kind="quicksort")
    inv = np.empty(sorter.size, dtype=np.intp)
    inv = inv_sorter(inv,sorter)
    arr = arr[sorter]
    obs = np.r_[True, arr[1:] != arr[:-1]]
    dense = summ(obs)[inv]
    count = np.r_[np.nonzero(obs)[0], len(obs)]
    return av(count,dense)


@numba.njit(cache=True)
def inv_sorter(inv,sorter):
    inv[sorter] = np.arange(sorter.size)
    return inv

@numba.njit(cache=True)
def av(count, dense):
    return .5 * (count[dense] + count[dense - 1] + 1)

def av_nojit(count,dense):
    return .5 * (count[dense] + count[dense - 1] + 1)


def no_jit_inv_sorter(inv,sorter):
    inv[sorter] = np.arange(sorter.size)
    return inv


In [66]:
timeit av(np.random.randn(1000000), np.random.randint(100))

63.4 ms ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [67]:
timeit av_nojit(np.random.randn(1000000), np.random.randint(100))

60.8 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [68]:
x = np.arange(1,10000,np.random.randint(100))
x


array([   1,   47,   93,  139,  185,  231,  277,  323,  369,  415,  461,
        507,  553,  599,  645,  691,  737,  783,  829,  875,  921,  967,
       1013, 1059, 1105, 1151, 1197, 1243, 1289, 1335, 1381, 1427, 1473,
       1519, 1565, 1611, 1657, 1703, 1749, 1795, 1841, 1887, 1933, 1979,
       2025, 2071, 2117, 2163, 2209, 2255, 2301, 2347, 2393, 2439, 2485,
       2531, 2577, 2623, 2669, 2715, 2761, 2807, 2853, 2899, 2945, 2991,
       3037, 3083, 3129, 3175, 3221, 3267, 3313, 3359, 3405, 3451, 3497,
       3543, 3589, 3635, 3681, 3727, 3773, 3819, 3865, 3911, 3957, 4003,
       4049, 4095, 4141, 4187, 4233, 4279, 4325, 4371, 4417, 4463, 4509,
       4555, 4601, 4647, 4693, 4739, 4785, 4831, 4877, 4923, 4969, 5015,
       5061, 5107, 5153, 5199, 5245, 5291, 5337, 5383, 5429, 5475, 5521,
       5567, 5613, 5659, 5705, 5751, 5797, 5843, 5889, 5935, 5981, 6027,
       6073, 6119, 6165, 6211, 6257, 6303, 6349, 6395, 6441, 6487, 6533,
       6579, 6625, 6671, 6717, 6763, 6809, 6855, 69

In [69]:
timeit inv_sorter(np.random.randn(1000000),x)

61.6 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [70]:
timeit no_jit_inv_sorter(np.random.randn(1000000),x)

61.1 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
lp = LineProfiler()
wrapper = lp(rankdata_new)
wrapper(numpy_data)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.745453 s
File: <ipython-input-65-669031228bc7>
Function: rankdata_new at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def rankdata_new(arr):
     2         1         33.0     33.0      0.0      arr = np.ravel(arr)
     3         1     255566.0 255566.0     34.3      sorter = np.argsort(arr, kind="quicksort")
     4         1         26.0     26.0      0.0      inv = np.empty(sorter.size, dtype=np.intp)
     5         1      47078.0  47078.0      6.3      inv = inv_sorter(inv,sorter)
     6         1      17027.0  17027.0      2.3      arr = arr[sorter]
     7         1       1746.0   1746.0      0.2      obs = np.r_[True, arr[1:] != arr[:-1]]
     8         1     362105.0 362105.0     48.6      dense = summ(obs)[inv]
     9         1       4342.0   4342.0      0.6      count = np.r_[np.nonzero(obs)[0], len(obs)]
    10         1      57530.0  57530.0      7.7      return av(cou

In [72]:
x = np.ravel(numpy_data)
x.shape

(1000000,)

In [73]:
%timeit summ(np.random.randn(100))

The slowest run took 4.74 times longer than the fastest. This could mean that an intermediate result is being cached.
14 µs ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [74]:
%timeit np.cumsum(np.random.randn(100))

13.3 µs ± 765 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [75]:
np.alltrue(summ(x)==np.cumsum(x))

True

In [76]:
%timeit rankdata_new(x)

358 ms ± 19.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
%timeit stats.rankdata(x)

362 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [78]:
school = az.load_arviz_data("centered_eight").posterior["mu"].values

In [79]:
%timeit rankdata_new(school)

304 µs ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [80]:
%timeit stats.rankdata(school)

383 µs ± 18.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [81]:
np.alltrue(rankdata_new(school)==stats.rankdata(school))

True

In [82]:
rankdata_new(x)

array([576575.,  20145., 400215., ..., 718170., 902640., 397462.])

In [83]:
np.alltrue(stats.rankdata(x)==rankdata_new(x))

True

In [84]:
def _z_scale_new(ary):
    ary = np.asarray(ary)
    size= ary.size
    rank = rankdata_new(ary)
    z = stats.norm.ppf((rank - 0.5) / size)
    z = z.reshape(ary.shape)
    return z

In [85]:
%timeit _z_scale_new(numpy_data)

515 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [86]:
%timeit _z_scale(numpy_data)

562 ms ± 37.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [87]:
np.allclose(_z_scale_new(numpy_data),_z_scale(numpy_data))

True

In [88]:
%timeit _z_scale(az.load_arviz_data("centered_eight").sample_stats.log_likelihood)

120 ms ± 8.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [89]:
%timeit _z_scale_new(az.load_arviz_data("centered_eight").sample_stats.log_likelihood)

124 ms ± 7.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [90]:
%timeit _z_scale(az.load_arviz_data("centered_eight").posterior["mu"].values)

112 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [91]:
%timeit _z_scale_new(az.load_arviz_data("centered_eight").posterior["mu"].values)

120 ms ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [92]:
'''''''''''''''''''''''''''''''''''z_scale works well on large sets. It's a bit slow on schools though'''''''''''''''

"''z_scale works well on large sets. It's a bit slow on schools though"

In [93]:
#RHAT

In [94]:
@numba.njit
def _var_1d(data):
    a,b = 0,0
    for i in data:
        a = a+i
        b = b+i*i
    return b/(len(data))-((a/(len(data)))**2)

@numba.njit
def _var_2d(data):
    a,b = data.shape
    var = np.zeros(a)
    for i in range(0,a):
        var[i] = _var_1d(data[i,:])
    return var



def _rhat_new(ary):
    ary = np.asarray(ary, dtype=float)
    if _not_valid(ary, check_shape=False):
        return np.nan
    _, num_samples = ary.shape

    # Calculate chain mean
    chain_mean = np.mean(ary, axis=1)
    # Calculate chain variance
    chain_var = _var_2d(ary)
    # Calculate between-chain variance
    between_chain_variance = num_samples * _var_1d(chain_mean)
    # Calculate within-chain variance
    within_chain_variance = np.mean(chain_var)
    # Estimate of marginal posterior variance
    rhat_value = np.sqrt(
        (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
    )
    return rhat_value

In [95]:
_rhat(numpy_data)

0.9999832532382178

In [96]:
 _rhat_new(numpy_data)

0.9999832532382178

In [97]:
school = az.load_arviz_data("centered_eight").posterior["mu"].values

In [98]:
_rhat(school)

1.01563316512969

In [99]:
_rhat_new(school)

1.0115252895535172

In [100]:
def _rhat_rank_new(ary):
    ary = np.asarray(ary)
    if _not_valid(ary, shape_kwargs=dict(min_draws=4, min_chains=2)):
        return np.nan
    split_ary = _split_chains(ary)
    rhat_bulk = _rhat_new(_z_scale_new(split_ary))

    split_ary_folded = abs(split_ary - np.median(split_ary))
    rhat_tail = _rhat_new(_z_scale_new(split_ary_folded))

    rhat_rank = max(rhat_bulk, rhat_tail)
    return rhat_rank


In [101]:
np.var(school,1)

array([10.37711934, 12.14856451, 11.91083021, 12.97533472])

In [102]:
_var_2d(school)

array([10.37711934, 12.14856451, 11.91083021, 12.97533472])

In [103]:
 _rhat_new(school)

1.0115252895535172

In [104]:
_rhat(school)

1.01563316512969

In [105]:
%timeit _rhat_rank_new(numpy_data)

1.17 s ± 68.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [106]:
%timeit rk(numpy_data)

1.2 s ± 80.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [107]:
timeit _rhat_rank_new(school)

2.38 ms ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [108]:
timeit rk(school)

2.98 ms ± 328 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [109]:
def _z_fold(ary):
    """Fold and z-scale values."""
    ary = np.asarray(ary)
    ary = abs(ary - np.median(ary))
    ary = _z_scale_new(ary)
    return ary


def _rhat_folded_new(ary):
    """Calculate split-Rhat for folded z-values."""
    ary = np.asarray(ary)
    if _not_valid(ary, shape_kwargs=dict(min_draws=4, min_chains=2)):
        return np.nan
    ary = _z_fold(_split_chains(ary))
    return _rhat_new(ary)


def _rhat_z_scale_new(ary):
    ary = np.asarray(ary)
    if _not_valid(ary, shape_kwargs=dict(min_draws=4, min_chains=2)):
        return np.nan
    return _rhat_new(_z_scale_new(_split_chains(ary)))


def _rhat_split_new(ary):
    ary = np.asarray(ary)
    if _not_valid(ary, shape_kwargs=dict(min_draws=4, min_chains=2)):
        return np.nan
    return _rhat_new(_split_chains(ary))


def _rhat_identity_new(ary):
    ary = np.asarray(ary)
    if _not_valid(ary, shape_kwargs=dict(min_draws=4, min_chains=2)):
        return np.nan
    return _rhat_new(ary)


In [110]:
def rhat_new(data, *, var_names=None, method="rank"):

    methods = {
        "rank": _rhat_rank_new,
        "split": _rhat_split_new,
        "folded": _rhat_folded_new,
        "z_scale": _rhat_z_scale_new,
        "identity": _rhat_identity_new,
    }
    if method not in methods:
        raise TypeError(
            "R-hat method {} not found. Valid methods are:\n{}".format(
                method, "\n    ".join(methods)
            )
        )
    rhat_func = methods[method]

    if isinstance(data, np.ndarray):
        data = np.atleast_2d(data)
        if len(data.shape) < 3:
            return rhat_func(data)
        else:
            msg = (
                "Only uni-dimensional ndarray variables are supported."
                " Please transform first to dataset with `az.convert_to_dataset`."
            )
            raise TypeError(msg)

    dataset = convert_to_dataset(data, group="posterior")
    var_names = _var_names(var_names, dataset)

    dataset = dataset if var_names is None else dataset[var_names]

    ufunc_kwargs = {"ravel": False}
    func_kwargs = {}
    return _wrap_xarray_ufunc(
        rhat_func, dataset, ufunc_kwargs=ufunc_kwargs, func_kwargs=func_kwargs
    )



In [111]:
numpy_data = np.random.randn(10000,1000)
dict_data = {"posterior":numpy_data}

In [112]:
%timeit rh(numpy_data)

15.7 s ± 551 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [113]:
%timeit rhat_new(numpy_data)

15.8 s ± 1.09 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [114]:
%timeit rh(numpy_data, method='split')

242 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [115]:
%timeit rhat_new(numpy_data, method='split')

139 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [116]:
%timeit rh(numpy_data, method='folded')

9.52 s ± 1.24 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [117]:
%timeit rhat_new(numpy_data, method='folded')

8.96 s ± 100 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [118]:
%timeit rh(numpy_data, method='z_scale')

8.93 s ± 160 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [119]:
%timeit rhat_new(numpy_data, method='z_scale')

8.58 s ± 138 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [120]:
%timeit rh(numpy_data, method="identity")

176 ms ± 4.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [121]:
%timeit rhat_new(numpy_data, method="identity")

63.3 ms ± 2.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [122]:
%timeit rh(school)

3.38 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [123]:
%timeit rhat_new(school)

2.9 ms ± 149 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [124]:
%timeit rh(school, method='split')

327 µs ± 22.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [125]:
%timeit rhat_new(school, method='split')

196 µs ± 19.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [126]:
%timeit rh(school, method='folded')

1.93 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [127]:
%timeit rhat_new(school, method='folded')

1.47 ms ± 92.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [128]:
%timeit rh(school, method='z_scale')

1.38 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [129]:
%timeit rhat_new(school, method='z_scale')

1.19 ms ± 55 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [130]:
%timeit rh(school, method='identity')

255 µs ± 24.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [131]:
%timeit rhat_new(school, method='identity')

154 µs ± 17.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [132]:
# Good Improvement in rhat 

In [133]:
"""""""""""""""""""""""""""""""""""""""""""""""'ESS"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

'""\'ESS'

In [134]:
data = np.random.randn(1000,1000)

In [135]:
@numba.njit
def loop_lifter(n_chain,n_draw,acov,chain_mean, mean_var, var_plus,rho_hat_t,rho_hat_even,rho_hat_odd,relative):
    t = 1
    while t < (n_draw - 3) and (rho_hat_even + rho_hat_odd) > 0.0:
        rho_hat_even = 1.0 - (mean_var - np.mean(acov[:, t + 1])) / var_plus
        rho_hat_odd = 1.0 - (mean_var - np.mean(acov[:, t + 2])) / var_plus
        if (rho_hat_even + rho_hat_odd) >= 0:
            rho_hat_t[t + 1] = rho_hat_even
            rho_hat_t[t + 2] = rho_hat_odd
        t += 2

    max_t = t - 2
    # improve estimation
    if rho_hat_even > 0:
        rho_hat_t[max_t + 1] = rho_hat_even
    # Geyer's initial monotone sequence
    t = 1
    while t <= max_t - 2:
        if (rho_hat_t[t + 1] + rho_hat_t[t + 2]) > (rho_hat_t[t - 1] + rho_hat_t[t]):
            rho_hat_t[t + 1] = (rho_hat_t[t - 1] + rho_hat_t[t]) / 2.0
            rho_hat_t[t + 2] = rho_hat_t[t + 1]
        t += 2

    ess = n_chain * n_draw
    tau_hat = -1.0 + 2.0 * np.sum(rho_hat_t[: max_t + 1]) + np.sum(rho_hat_t[max_t + 1 : max_t + 2])
    tau_hat = max(tau_hat, 1 / np.log10(ess))
    ess = (1 if relative else ess) / tau_hat
    if np.isnan(rho_hat_t).any():
        ess = np.nan
    return ess

In [136]:

def _ess_new(ary, relative=False):
    """Compute the effective sample size for a 2D array."""
    ary = np.asarray(ary, dtype=float)
    if _not_valid(ary, check_shape=False):
        return np.nan
    if (np.max(ary) - np.min(ary)) < np.finfo(float).resolution:  # pylint: disable=no-member
        return ary.size
    if len(ary.shape) < 2:
        ary = np.atleast_2d(ary)
    n_chain, n_draw = ary.shape
    acov = _autocov(ary, axis=1)
    chain_mean = ary.mean(axis=1)
    mean_var = np.mean(acov[:, 0]) * n_draw / (n_draw - 1.0)
    var_plus = mean_var * (n_draw - 1.0) / n_draw
    if n_chain > 1:
        var_plus += np.var(chain_mean, ddof=1)

    rho_hat_t = np.zeros(n_draw)
    rho_hat_even = 1.0
    rho_hat_t[0] = rho_hat_even
    rho_hat_odd = 1.0 - (mean_var - np.mean(acov[:, 1])) / var_plus
    rho_hat_t[1] = rho_hat_odd
    
    ess = loop_lifter(n_chain,n_draw,acov,chain_mean, mean_var, var_plus,rho_hat_t,rho_hat_even,rho_hat_odd,relative)
    return ess

    # Geyer's initial positive sequence
    


In [137]:
np.allclose(_ess_new(data), es(data))

True

In [138]:
%timeit _ess_new(data)

61.8 ms ± 5.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [139]:
%timeit es(data)

46.2 ms ± 806 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [140]:
# Ess cannot be improved further.There is no point in testing other _ess methods as all of them involve _ess func at one point or another


In [141]:
# MCSE

In [142]:
data = np.random.randn(1000,1000)
school = az.load_arviz_data("centered_eight").posterior["mu"].values

In [143]:
lp = LineProfiler()
wrapper = lp(_mcse_mean)
wrapper(data)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.057905 s
File: /home/banzee/Desktop/arviz/arviz/stats/diagnostics.py
Function: _mcse_mean at line 860

Line #      Hits         Time  Per Hit   % Time  Line Contents
   860                                           def _mcse_mean(ary):
   861                                               """Compute the Markov Chain mean error."""
   862         1          8.0      8.0      0.0      ary = np.asarray(ary)
   863         1       1765.0   1765.0      3.0      if _not_valid(ary, shape_kwargs=dict(min_draws=4, min_chains=1)):
   864                                                   return np.nan
   865         1      49014.0  49014.0     84.6      ess = _ess_mean(ary)
   866         1       7109.0   7109.0     12.3      sd = np.std(ary, ddof=1)
   867         1          8.0      8.0      0.0      mcse_mean_value = sd / np.sqrt(ess)
   868         1          1.0      1.0      0.0      return mcse_mean_value



In [144]:
timeit np.sqrt(_var_2d(data))

1.95 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [145]:
timeit np.std(data,axis=1)

7.59 ms ± 747 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [146]:

np.allclose(np.sqrt(_var_2d(data)),np.std(data,axis=1))

True

In [147]:
def _mcse_mean_new(ary):
    ary = np.asarray(ary)
    if _not_valid(ary, shape_kwargs=dict(min_draws=4, min_chains=1)):
        return np.nan
    ess = _ess_mean(ary)
    ary = np.ravel(ary)
    sd = np.sqrt(_var_1d(ary))
    mcse_mean_value = sd / np.sqrt(ess)
    return mcse_mean_value

In [148]:
_mcse_mean(data)/_mcse_mean_new(data)

1.0000005000003742

In [149]:
_mcse_mean(school)/_mcse_mean_new(school)

1.000250093789082

In [150]:
np.sqrt(_var_1d(np.ravel(data)))

1.000674047731067

In [151]:
np.std(data,ddof=1)

1.0006745480684656

In [152]:
np.sqrt(_var_1d(np.ravel(school)))

3.4858944648051677

In [153]:
np.std(school,ddof=1)

3.4867662653602105

In [154]:
%timeit _mcse_mean(data)

71.1 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [155]:
%timeit _mcse_mean_new(data)

69.7 ms ± 3.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [156]:
%timeit _mcse_mean_new(school)

2.04 ms ± 230 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [157]:
%timeit _mcse_mean(school)

2.27 ms ± 362 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [158]:
def _mcse_sd_new(ary):
    """Compute the Markov Chain sd error."""
    ary = np.asarray(ary)
    if _not_valid(ary, shape_kwargs=dict(min_draws=4, min_chains=1)):
        return np.nan
    ess = _ess_sd(ary)
    ary = np.ravel(ary)
    sd = np.sqrt(_var_1d(ary))
    fac_mcse_sd = np.sqrt(np.exp(1) * (1 - 1 / ess) ** (ess - 1) - 1)
    mcse_sd_value = sd * fac_mcse_sd
    return mcse_sd_value

In [159]:
_mcse_sd_new(data)

0.0007068803404174883

In [160]:
msd(data)

0.0007068806938579233

In [161]:
_mcse_sd_new(school)

0.1856619288742948

In [162]:
msd(school)

0.18570836176957523

In [163]:
%timeit _mcse_sd_new(data)

126 ms ± 5.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [164]:
%timeit msd(data)

137 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [165]:
%timeit _mcse_sd_new(school)

4.08 ms ± 217 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [166]:
%timeit msd(school)

4.39 ms ± 289 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [167]:
def _circfuncs_common(samples, high, low):
    samples = np.asarray(samples)
    if samples.size == 0:
        return np.nan, np.nan
    return samples, angle(samples, low,high,np.pi)

@numba.vectorize(nopython=True)
def angle(samples,low,high,pi=np.pi):
    ang = (samples - low)*2.*pi / (high - low)
    return ang

In [168]:
def _circular_standard_deviation(samples, high=2*np.pi, low=0, axis=None):
    pi = np.pi
    samples, ang = _circfuncs_common(samples, high, low)
    S = np.sin(ang).mean(axis=axis)
    C = np.cos(ang).mean(axis=axis)
    R = np.hypot(S, C)
    return ((high - low)/2.0/pi) * np.sqrt(-2*np.log(R))

In [169]:
lp = LineProfiler()
wrapper = lp(me)
wrapper(data,20,True)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.328963 s
File: /home/banzee/Desktop/arviz/arviz/stats/diagnostics.py
Function: _mc_error at line 892

Line #      Hits         Time  Per Hit   % Time  Line Contents
   892                                           def _mc_error(ary, batches=5, circular=False):
   893                                               """Calculate the simulation standard error, accounting for non-independent samples.
   894                                           
   895                                               The trace is divided into batches, and the standard deviation of the batch
   896                                               means is calculated.
   897                                           
   898                                               Parameters
   899                                               ----------
   900                                               ary : Numpy array
   901                                                   An array 

In [170]:
def _mc_error_new(ary, batches=5, circular=False):
    if ary.ndim > 1:

        dims = np.shape(ary)
        trace = np.transpose([t.ravel() for t in ary])

        return np.reshape([_mc_error(t, batches) for t in trace], dims[1:])

    else:
        if _not_valid(ary, check_shape=False):
            return np.nan
        if batches == 1:
            if circular:
                std = _circular_standard_deviation(ary, high=np.pi, low=-np.pi)
            else:
                std = np.sqrt(_var_1d(ary))
            return std / np.sqrt(len(ary))

        batched_traces = np.resize(ary, (batches, int(len(ary) / batches)))

        if circular:
            means = stats.circmean(batched_traces, high=np.pi, low=-np.pi, axis=1)
            std = _circular_standard_deviation(means, high=np.pi, low=-np.pi)
        else:
            means = np.mean(batched_traces, 1)
            std = np.sqrt(_var_1d(means))

        return std / np.sqrt(batches)


In [172]:
%timeit _mc_error_new(data, circular=True)

NameError: name '_mc_error' is not defined

In [ ]:
%timeit me(data, circular=True)

In [ ]:
%timeit _mc_error_new(school, circular=True)

In [ ]:
%timeit me(school, circular=True)

In [ ]:
##############################################DONE####################################################################

In [178]:
data = np.random.randn(100001)

In [209]:
@numba.njit
def med(data):
    x = len(data)
    for i in range(0,x-1):
        for j in range(0,x-i):
            if data[i]<data[j]:
                temp = data[i]
                data[i]=data[j]
                data[j]=temp
    if x%2 != 0:
        return data[(x-1)//2]
    else:
        x = x//2
        return (data[x]+data[x-1])/2

In [210]:
%timeit med(data)

5.37 s ± 5.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [211]:
%timeit np.median(data)

1.2 ms ± 58.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [193]:
med(data)==np.median(data)

True